In [1]:

from extracthero.utils import prepare_numbered_content, read_md
from extracthero import FilterHero, WhatToRetain
from extracthero.myllmservice import MyLLMService

from extracthero import FilterHero, WhatToRetain

filter_hero = FilterHero()


In [2]:

spec1 = WhatToRetain(
        name="core_api_documentation",
        desc="""
                api description, endpoints, code examples (all available languages), Request Details, Response Details,
                Data Models ,Authentication & Authorization logic ,Error Handling 

            """ )


spec2 = WhatToRetain(
      name="core_api_documentation",
      desc="api endpoints, code examples, requests, responses"
  )



# name="api_technical_implementation",
#       desc="Actual technical instrictions, endpoint URLs, request bodies, response formats, authentication code and all sort of code samples",

spec3 = WhatToRetain(
      name="api_technical_implementation",
      desc="Actual technical instrictions, endpoint URLs, request bodies, response formats, authentication code and all sort of code samples",
      text_rules=[
            
            "This is a SINGLE PAGE extraction - all cross-references are irrelevant",
            "any line that is primarily a hyperlink to another resource is irrelevant", 
            "all navigation menus and UI elements are irrelevant"
            

      ]
  )

spec4 = WhatToRetain(
      name="api_technical_implementation",
      desc="Actual technical instrictions, endpoint URLs, request bodies, response formats, authentication code and all sort of code samples",
      text_rules=[
            
            "This is a SINGLE PAGE extraction - all cross-references are irrelevant",
            "any line that is primarily a hyperlink to another resource is irrelevant", 
            "all navigation menus and UI elements are irrelevant"
            

      ]
  )


In [3]:


# sample_md_path = "samples/page_001_developer.adobe.com_firefly-services_docs_firefly-api_guides_.md"
sample_md_path = "samples/2.md"
sample_md=read_md(sample_md_path)

# Show info about the loaded content
print(f"✅ Loaded {len(sample_md)} characters from {sample_md_path}")
print(f"✅ Number of lines: {len(sample_md.splitlines())}")

original_lines = sample_md.split('\n')
numbered_content = prepare_numbered_content( original_lines )

✅ Loaded 17099 characters from samples/2.md
✅ Number of lines: 538


In [ ]:
FILTER = """
You are reviewing numbered content to identify ALL irrelevant sections for deletion.

CORPUS:
{numbered_corpus}

CONTENT TO PRESERVE:
{thing_to_extract}


You are given a CORPUS which is a web page content in markdown. 
And your task is to inspect this corpus line by line and identify irrelevant parts with respect to given CONTENT TO PRESERVE,  In such format:


OUTPUT FORMAT (JSON only):
{{
  "deletions": [
    {{"start_line": X, "end_line": Y}}
  ]
}}


you can ignore blank lines if previos or 

Output ONLY valid JSON. Be thorough and identify all irrelevant content.
"""
 
 
user_prompt = FILTER.format(
                    numbered_corpus=numbered_content,
                    thing_to_extract=spec3
                )



# OUTPUT FORMAT (JSON only):
# {{
#   "deletions": [
#     {{"start_line": X, "end_line": Y, "reason": "category"}}
#   ]
# }}


#  numbered content to identify ALL irrelevant sections for deletion line by line.
# So inspect each line and identify the lines or line groups which are not directly related to given 
# CONTENT TO PRESERVE above



In [ ]:
# #  "This is a SINGLE PAGE extraction - all cross-references are irrelevant",
# #  "any line that is primarily a hyperlink to another resource is irrelevant",
# #       "Links in format [text](url) or [text][ref] should be removed",
# #       "We need the ACTUAL content, not the TABLE OF CONTENTS",
# #       "If the information isn't written out HERE, we don't want it",
# #       "Navigation saying 'Rate limits' with a link is NOT rate limit documentation",
# #       "Only keep 'Rate limits' if followed by actual rate limit values/rules"
# #   ]

#   Most direct phrasing:

#   "If it's just a link pointing somewhere else, DELETE IT. We only want information that's actually written on THIS page."

In [ ]:
FILTER = """
You are analyzing a web page content in order to identify non-target information. So we can remove them based on your output

HERE IS Target Information we are looking for:

{thing_to_extract}

HERE IS WEB PAGE Content in Markdown:
________________START________________
{numbered_corpus}
________________END________________

inspect this markdown corpus and  Identify NON-ESSENTIAL sections that don't contain actual implementation details  by their Line number, 

- Sections Primarily Containing Hyperlinks to Other Resources are IRRELEVANT -
- Dont give explanation just give which line periods which are non essential- 


OUTPUT FORMAT (JSON only):
        {{
       "deletions": [
     
       {{"start": X, "end": Y}}
       ]
      }}


"""
 
  #dont give explanation just give which line periods are irrelevant
 
user_prompt = FILTER.format(
                    numbered_corpus=numbered_content,
                    thing_to_extract=spec3
                )









In [ ]:
# FILTER = """
# You are reviewing numbered content to identify ALL irrelevant sections for deletion.

# CORPUS:
# {numbered_corpus}

# CONTENT TO PRESERVE:
# {thing_to_extract}


# You are given a CORPUS which is a web page content in markdown. And your task is to inspect this markdown corpus and identify  (with respect to given CONTENT TO PRESERVE) irrelevant markdown sections. 

# And output the results In such format:


# OUTPUT FORMAT (JSON only):
# {{
#   "deletions": [
#     {{"start_line": X, "end_line": Y}}
#   ]
# }}


# Aim to output big chunks/sections rather then just one line,

# Output ONLY valid JSON. Be thorough and identify all irrelevant content.
# """
 
 
# user_prompt = FILTER.format(
#                     numbered_corpus=numbered_content,
#                     thing_to_extract=spec3
#                 )





In [ ]:
# FILTER = """
# You are reviewing numbered content to identify ALL irrelevant sections for deletion.

# CORPUS:
# {numbered_corpus}

# CONTENT TO PRESERVE:
# {thing_to_extract}


# You are given a CORPUS which is a web page content in markdown. And your task is to inspect this markdown corpus and identify  (with respect to given CONTENT TO PRESERVE) irrelevant markdown sections. 

# And output the results In such format:


# OUTPUT FORMAT (JSON only):
# {{
#   "deletions": [
#     {{"start_line": X, "end_line": Y}}
#   ]
# }}


# you can ignore blank lines. Aim to output big chunks/sections rather then just one line

# Output ONLY valid JSON. Be thorough and identify all irrelevant content.
# """
 
 
# user_prompt = FILTER.format(
#                     numbered_corpus=numbered_content,
#                     thing_to_extract=spec
#                 )





In [8]:
my_llmservice= MyLLMService()


gen_result= my_llmservice.get_deletions_via_llm_custom(user_prompt, model="gpt-4o" )
# data= gen_result.content.get('deletions', [])





# for d in data:
#     print(d)

In [9]:
gen_result.raw_content

'{"deletions":[\n\n   { \t\t\t\n  \t\t \t}\t\n  \t]\n \t,"total_lines_to_delete":null,"reasoning":null}'

In [ ]:
gen_result.raw_content

In [ ]:
my_llmservice= MyLLMService()


gen_result= my_llmservice.get_deletions_via_llm_custom(user_prompt, model="gpt-4o" )
data= gen_result.content.get('deletions', [])


for d in data:
    print(d)

In [ ]:

my_llmservice= MyLLMService()


gen_result= my_llmservice.get_deletions_via_llm_custom(user_prompt, model="gpt-4o-mini" )
data= gen_result.content.get('deletions', [])


for d in data:
    print(d)


In [ ]:

my_llmservice= MyLLMService()


gen_result= my_llmservice.get_deletions_via_llm_custom(user_prompt, model="gpt-4o" )
data= gen_result.content.get('deletions', [])


for d in data:
    print(d)




